# Predicting LendingClub Loan Charge-offs from Initial Listing Data


**June 14, 2018**

## Table of Contents

* [1. Introduction](#1)
 * [1.1 References](#1.1)
* [2. Import the Data](#2)
* [3. Response Variable](#3)
* [4. Limit the Feature Space](#4)
 * [4.1 Drop features missing more than 30% data](#4.1)
 * [4.2 Only keep loan features known to potential investors](#4.2)
* [5. Pre-processing and Exploratory Analysis](#5)
* [6. More Pre-processing](#6)
 * [6.1 Convert loan status to 0/1 charge-off indicator](#6.1)
 * [6.2 Create dummy variables](#6.2)
 * [6.3 Train/test split](#6.3)
* [7. Linear Dependence of Charge-off on the Predictors](#7)
* [8. Model Training and Testing](#8)
 * [8.1 Logistic regression with SGD training](#8.1)
 * [8.2 Random forest classifier](#8.2)
 * [8.3 k-nearest neighbors](#8.3)
 * [8.4 Tune hyperparameters on the chosen model more finely](#8.4)
 * [8.5 Test set evaluation](#8.5)
* [9. Conclusion](#9)

* There are new powerful machine learning models:
 1 CatBoost 2 XGBoost, 3 LightGBM

# 1. Introduction
<a id="1"></a>

[LendingClub](https://www.lendingclub.com/) is a US peer-to-peer lending company and the world's largest peer-to-peer lending platform. As explained by [Wikipedia](https://en.wikipedia.org/wiki/Lending_Club),

> Lending Club enables borrowers to create unsecured personal loans between \$1,000 and \$40,000. The standard loan period is three years. Investors can search and browse the loan listings on Lending Club website and select loans that they want to invest in based on the information supplied about the borrower, amount of loan, loan grade, and loan purpose. Investors make money from interest. Lending Club makes money by charging borrowers an origination fee and investors a service fee.

The goal of this project is to build a machine learning model to predict the probability that a loan will charge off. We will attempt to only use data available to investors via the LendingClub loan listing, including information about the borrower (income, employment length, FICO score, debt-to-income ratio, etc.) and the loan listing (the loan amount, loan purpose, loan grade, interest rate, installment, etc.). Such a predictive model could help LendingClub investors make better-informed investment decisions. We will only consider loans that LendingClub accepted under its credit underwriting policy.

## 1.1 References
<a id="1.1"></a>

* LendingClub information
 * [LendingClub website](https://www.lendingclub.com/)
 * [LendingClub Wikipedia page](https://en.wikipedia.org/wiki/Lending_Club)
* Datasets
 * [LendingClub statistics](https://www.lendingclub.com/info/download-data.action) - Original data source, aggregated on Kaggle
 * [All Lending Club loan data](https://www.kaggle.com/wordsforthewise/lending-club) - The dataset used in this project, hosted on Kaggle
 * [Lending Club Loan Data](https://www.kaggle.com/wendykan/lending-club-loan-data) - Another LendingClub dataset on Kaggle, not used in this project


# 2. Import the Data
<a id="2"></a>

In [ ]:
import numpy as npseaborn
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Pandas options
pd.set_option('display.max_colwidth', 1000, 'display.max_rows', None, 'display.max_columns', None)

# Plotting options
%matplotlib inline
mpl.style.use('ggplot')
sns.set(style='whitegrid')
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


Read the data into a pandas dataframe:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
loans = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/MGMT635 Project 2/200000.csv')


In [ ]:
loans.shape

(199999, 151)

In [ ]:
import pandas as pd
from google.colab import files
import io

# 1. Initiate upload
uploaded = files.upload()

# 2. Get the filename and read the content directly from memory
file_name = '200000.csv'
loans = pd.read_csv(io.BytesIO(uploaded[file_name]))

# 3. Verify the load worked
print(loans.shape)
print(loans.info())

Check basic dataframe info:

In [ ]:
loans.info()

For the complete dataset, there are 1.6 million rows and 150 variables. The size of the dataset is 1.8 GB.

Let's peek at 5 randomly selected rows. Each row corresponds to a single loan.

In [ ]:
smallerSample= loans.sample(1000)

In [ ]:
help(loans.sample)

In [ ]:
smallerSample.info()


In [ ]:
smallerSample['desc']

# 3. Response Variable
<a id="3"></a>

We're going to try to predict the `loan_status` variable. What are the value counts for this variable?

In [ ]:
loans['loan_status'].value_counts(dropna=False)

We're going to try to learn differences in the features between completed loans that have been fully paid or charged off. We won't consider loans that are current, don't meet the credit policy, defaulted, or have a missing status. So we only keep the loans with status "Fully Paid" or "Charged Off."

In [ ]:
loans = loans.loc[loans['loan_status'].isin(['Fully Paid', 'Charged Off'])]

How many loans remain in the dataset?

In [ ]:
loans.shape

There are 175421 loans remaining. Check that the statuses are as expected:

In [ ]:
loans['loan_status'].value_counts(dropna=False)

Status counts as percentages:

In [ ]:
loans['loan_status'].value_counts(normalize=True, dropna=False)

About 79% of the remaining loans have been fully paid and 20% have charged off, so we have a somewhat unbalanced classification problem.

# 4. Limit the Feature Space
<a id="4"></a>

The full dataset has 150 features for each loan. We'll select features in two steps:

1. Drop features with more than 30% of their data missing.
2. Of the remaining features, choose only those that would be available to an investor before deciding to fund the loan.

Definitions of the features are given in the LendingClub Data Dictionary [available here](https://www.lendingclub.com/info/download-data.action).

## 4.1 Drop features missing more than 30% data
<a id="4.1"></a>

First calculate the percentage of missing data for each feature:

In [ ]:
missing_fractions = loans.isnull().mean().sort_values(ascending=False)


In [ ]:
missing_fractions

Top 10 features missing the most data:

In [ ]:
len(missing_fractions)

Let's visualize the distribution of missing data percentages:

In [ ]:
plt.figure(figsize=(6,3), dpi=90)
missing_fractions.plot.hist(bins=20)
plt.title('Histogram of Feature Incompleteness')
plt.xlabel('Fraction of data missing')
plt.ylabel('Feature count')

From the above histogram, we see there's a large gap between features missing "some" data (&lt;20%) and those missing "lots" of data (&gt;40%). Because it's generally very difficult to accurately impute data with more than 30% missing values, we drop such columns. First store all variables missing more than 30% data in an alphabetical list:

In [ ]:
drop_list = sorted(list(missing_fractions[missing_fractions > 0.3].index))
print(drop_list)

How many features will be dropped?

In [ ]:
len(drop_list)

Drop these features:

In [ ]:
loans.drop(labels=drop_list, axis=1, inplace=True)

In [ ]:
loans.shape

## 4.2 Only keep loan features known to potential investors
<a id="4.2"></a>

We examine the LendingClub website and Data Dictionary to determine which features would have been available to potential investors. Here's the list of features we currently have, in alphabetical order:

In [ ]:
print(sorted(loans.columns))

For each of these features, we check the description in the Data Dictionary and only keep the features that would have been available to investors considering an investment in the loan. These include features in the loan application, and any features added by LendingClub when the loan listing was accepted, such as the loan grade and interest rate.

I'm using my best available knowledge to determine which loan features are known to potential investors. I am not an investor on LendingClub, so my knowledge of the LendingClub investment process is not exact. When in doubt, I err on the side of dropping the feature.

In [ ]:
keep_list = ['addr_state', 'annual_inc', 'application_type', 'dti', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade', 'home_ownership', 'id', 'initial_list_status', 'installment', 'int_rate', 'issue_d', 'loan_amnt', 'loan_status', 'mort_acc', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 'total_acc', 'verification_status', 'zip_code']

In [ ]:
len(keep_list)

The list of features to drop is any feature not in `keep_list`:

In [ ]:
drop_list = [col for col in loans.columns if col not in keep_list]
print(drop_list)

In [ ]:
len(drop_list)

Drop these features:

In [ ]:
loans.drop(labels=drop_list, axis=1, inplace=True)

In [ ]:
loans.shape

# 5. Data Understanding and Exploratory Analysis
<a id="5"></a>

We'll inspect each feature individually, and do the following:

1. Drop the feature if it is not useful for predicting the loan status.
2. View summary statistics and visualize the data, plotting against the loan status.
3. Modify the feature to make it useful for modeling, if necessary.

We define a function for plotting a variable and comparing with the loan status:

In [ ]:
def plot_var(col_name, full_name, continuous):
    """
    Visualize a variable with and without faceting on the loan status.
    - col_name is the variable name in the dataframe
    - full_name is the full variable name
    - continuous is True if the variable is continuous, False otherwise
    """
    f, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,3), dpi=90)

    # Plot without loan status
    if continuous:
        sns.distplot(loans.loc[loans[col_name].notnull(), col_name], kde=False, ax=ax1)
    else:
        sns.countplot(x=loans[col_name], order=sorted(loans[col_name].unique()), color='#5975A4', saturation=1, ax=ax1)
    ax1.set_xlabel(full_name)
    ax1.set_ylabel('Count')
    ax1.set_title(full_name)

    # Plot with loan status
    if continuous:
        sns.boxplot(x=col_name, y='loan_status', data=loans, ax=ax2)
        ax2.set_ylabel('')
        ax2.set_title(full_name + ' by Loan Status')
    else:
        charge_off_rates = loans.groupby(col_name)['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']
        sns.barplot(x=charge_off_rates.index, y=charge_off_rates.values, color='#5975A4', saturation=1, ax=ax2)
        ax2.set_ylabel('Fraction of Loans Charged-off')
        ax2.set_title('Charge-off Rate by ' + full_name)
    ax2.set_xlabel(full_name)
    plt.tight_layout()

Print the remaining features for future reference:

In [ ]:
print(list(loans.columns))

## 5.1 id

Data Dictionary: "A unique [LendingClub] assigned ID for the loan listing."

In [ ]:
loans.head(5)

In [ ]:
loans['id'].sample(5)

Are all the IDs unique?

In [ ]:
loans['id'].describe()

Yes, they are all unique. The ID is not useful for modeling, either as a categorical variable (there are too many distinct values) or as a numerical variable (the IDs vary wildly in magnitude, likely without any significance), so we drop this variable.

In [ ]:
loans.drop('id', axis=1, inplace=True)

## 5.2 loan_amnt

Data Dictionary: "The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value."

In [ ]:
loans['loan_amnt'].head()

In [ ]:
loans['loan_amnt'].describe()

Loan amounts range from \$500 to \$40,000, with a median of \$12,000.

In [ ]:
plot_var('loan_amnt', 'Loan Amount', continuous=True)

Charged-off loans tend to have higher loan amounts. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['loan_amnt'].describe()

## 5.3 term

Data Dictionary: "The number of payments on the loan. Values are in months and can be either 36 or 60."

In [ ]:
loans['term'].value_counts(dropna=False)

In [ ]:
loans['term'].head()

Convert `term` to integers.

In [ ]:
import numpy as np

loans['term'] = loans['term'].apply(lambda s: np.int8(s.split()[0]))

In [ ]:
loans['term'].value_counts(normalize=True)

Compare the charge-off rate by loan period:

In [ ]:
loans.groupby('term')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

About 76% of the completed loans have three-year periods, and the rest have five-year periods. Loans with five-year periods are more than twice as likely to charge-off as loans with three-year periods.

## 5.4 int_rate  Step for Data Understanding .


Data Dictionary: "Interest Rate on the loan."

In [ ]:
loans['int_rate'].describe()

Interest rates range from 5.32% to 30.99% (!) with a median of 13.1%.

In [ ]:
plot_var('int_rate', 'Interest Rate', continuous=True)

Charged-off loans tend to have much higher interest rates. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['int_rate'].describe()

## 5.5 installment

Data Dictionary: "The monthly payment owed by the borrower if the loan originates."

In [ ]:
loans['installment'].describe()

Installments range from \$4.93 to \$1,714, with a median of \$377.

In [ ]:
plot_var('installment', 'Installment', continuous=True)

Charged-off loans tend to have higher installments. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['installment'].describe()

Loans that charge off have \$30 higher installments on average.

## 5.6 grade, sub_grade

Data Dictionary for `grade`: "LendingClub assigned loan grade."

Data Dictionary for `sub_grade`: "LendingClub assigned loan subgrade."

What are the possible values of `grade` and `sub_grade`?

In [ ]:
print(sorted(loans['grade'].unique()))

In [ ]:
print(sorted(loans['sub_grade'].unique()))

The grade is implied by the subgrade, so let's drop the grade column.

In [ ]:
loans.drop('grade', axis=1, inplace=True)

In [ ]:
plot_var('sub_grade', 'Subgrade', continuous=False)

There's a clear trend of higher probability of charge-off as the subgrade worsens.

## 5.7 emp_title

Data Dictionary: "The job title supplied by the Borrower when applying for the loan."

In [ ]:
loans['emp_title'].describe()

There are too many different job titles for this feature to be useful, so we drop it.

In [ ]:
loans.drop(labels='emp_title', axis=1, inplace=True)

## 5.8 emp_length

Data Dictionary: "Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years." The actual data does not match this description:

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

In [ ]:
loans['emp_length'].head(100)

Note there are 42,253 loans without data on the length of employment.

Convert `emp_length` to integers:

In [ ]:
loans['emp_length'].replace(to_replace='10+ years', value='10 years', inplace=True)

In [ ]:
loans['emp_length'].replace('< 1 year', '0 years', inplace=True)

In [ ]:
def emp_length_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

In [ ]:
loans['emp_length'] = loans['emp_length'].apply(emp_length_to_int)

In [ ]:
loans['emp_length'].value_counts(dropna=False).sort_index()

In [ ]:
help(plot_var)

In [ ]:
plot_var('emp_length', 'Employment Length', continuous=False)

Loan status does not appear to vary much with employment length on average, except for a small drop in charge-offs for borrowers with over 10 years of employment.

## 5.9 home_ownership

Data Dictionary: "The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER."

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

Replace the values `ANY` and `NONE` with `OTHER`:

In [ ]:
loans['home_ownership'].replace(['NONE', 'ANY'], 'OTHER', inplace=True)

In [ ]:
loans['home_ownership'].value_counts(dropna=False)

In [ ]:
plot_var('home_ownership', 'Home Ownership', continuous=False)

There appear to be large differences in charge-off rates by home ownership status. Renters and homeowners have a higher probability of charge-off. Let's compare the charge-off rates:

In [ ]:
loans.groupby('home_ownership')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

## 5.10 annual_inc

Data Dictionary: "The self-reported annual income provided by the borrower during registration."

In [ ]:
loans['annual_inc'].describe()

Annual income ranges from \$0 to \$9,550,000, with a median of \$65,000. Because of the large range of incomes, we should take a log transform of the annual income variable.

In [ ]:
loans['log_annual_inc'] = loans['annual_inc'].apply(lambda x: np.log10(x+1))

In [ ]:
loans.drop('annual_inc', axis=1, inplace=True)

In [ ]:
loans['log_annual_inc'].describe()

In [ ]:
plot_var('log_annual_inc', 'Log Annual Income', continuous=True)

It appears that individuals with higher income are more likely to pay off their loans. Let's compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['log_annual_inc'].describe()

## 5.11 verification_status

Data Dictionary: "Indicates if income was verified by [Lending Club], not verified, or if the income source was verified."

In [ ]:
plot_var('verification_status', 'Verification Status', continuous=False)

## 5.12 issue_d

Data Dictionary: "The month which the loan was funded."

Because we're only using variables available to investors before the loan was funded, `issue_d` will not be included in the final model. We're keeping it for now just to perform the train/test split later, then we'll drop it.

## 5.13 purpose

Data Dictionary: "A category provided by the borrower for the loan request."

In [ ]:
loans['purpose'].value_counts()

Calculate the charge-off rates by purpose:

In [ ]:
loans.groupby('purpose')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values()

Notice that only 12% of completed loans for weddings have charged-off, but 30% of completed small business loans have charged-off.

## 5.14 title

Data Dictionary: "The loan title provided by the borrower."

In [ ]:
loans['title'].describe()

View the top 10 loan titles, and their frequencies:

In [ ]:
loans['title'].value_counts().head(10)

There are 60,298 different titles in the dataset, and based on the top 10 titles, the `purpose` variable appears to already contain this information. So we drop the `title` variable.

In [ ]:
loans.drop('title', axis=1, inplace=True)

## 5.15 zip_code, addr_state

Data Dictionary for `zip_code`: "The first 3 numbers of the zip code provided by the borrower in the loan application."

Data Dictionary for `addr_state`: "The state provided by the borrower in the loan application."

In [ ]:
loans['zip_code'].sample(5)

In [ ]:
loans['zip_code'].nunique()

In [ ]:
loans['addr_state'].sample(5)

In [ ]:
loans['addr_state'].nunique()

There are a lot of different zip codes, so let's just keep the state column.

In [ ]:
loans.drop(labels='zip_code', axis=1, inplace=True)

Calculate the charge-off rates by address state:

In [ ]:
loans.groupby('addr_state')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off'].sort_values()

The charge-off rate ranges from 13.0% in Washington, DC to 27.6% in Mississippi.

## 5.16 dti

Data Dictionary: "A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income."

In [ ]:
loans['dti'].describe()

Note sure if the values of -1 and 999 make sense...

There are several outliers that mess up our default plots. Plot a histogram for `dti` less than 60:

In [ ]:
plt.figure(figsize=(8,3), dpi=90)
sns.distplot(loans.loc[loans['dti'].notnull() & (loans['dti']<60), 'dti'], kde=False)
plt.xlabel('Debt-to-income Ratio')
plt.ylabel('Count')
plt.title('Debt-to-income Ratio')

How many of the `dti` values are "outliers" (above 60)?

In [ ]:
(loans['dti']>=60).sum()

Very few. Compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['dti'].describe()

Completed loans that are charged off tend to have higher debt-to-income ratios.

## 5.17 earliest_cr_line

Data Dictionary: "The month the borrower's earliest reported credit line was opened."

In [ ]:
loans['earliest_cr_line'].sample(5)

In [ ]:
loans['earliest_cr_line'].isnull().any()

Let's just retain the year for simplicity:

In [ ]:
loans['earliest_cr_line'] = loans['earliest_cr_line'].apply(lambda s: int(s[-4:]))

In [ ]:
loans['earliest_cr_line'].describe()

In [ ]:
plot_var('earliest_cr_line', 'Year of Earliest Credit Line', continuous=True)

Borrowers who are charged-off tend to have shorter lines of credit.

## 5.18 fico_range_low, fico_range_high

Data Dictionary for `fico_range_low`: "The lower boundary range the borrower’s FICO at loan origination belongs to."

Data Dictionary for `fico_range_high`: "The upper boundary range the borrower’s FICO at loan origination belongs to."

In [ ]:
loans[['fico_range_low', 'fico_range_high']].describe()

Check the Pearson correlation between these values:

In [ ]:
loans[['fico_range_low','fico_range_high']].corr()

In [ ]:
loans[['fico_range_low', 'fico_range_high']].corr()

In [ ]:
loans.head()

We only need to keep one of the FICO scores. We'll take the average of the two and call it `fico_score`:

In [ ]:
loans['fico_score'] = 0.5*loans['fico_range_low'] + 0.5*loans['fico_range_high']

In [ ]:
loans.drop(['fico_range_high', 'fico_range_low'], axis=1, inplace=True)

In [ ]:
plot_var('fico_score', 'FICO Score', continuous=True)

There is a noticeable difference in FICO scores between fully paid and charged-off loans. Compare the summary statistics:

In [ ]:
loans.groupby('loan_status')['fico_score'].describe()

Loans that charge off have a FICO score 10 points lower on average.

## 5.19 open_acc

Data Dictionary: "The number of open credit lines in the borrower's credit file."

In [ ]:
plt.figure(figsize=(10,3), dpi=90)
sns.countplot(x=loans['open_acc'], order=sorted(loans['open_acc'].unique()), color='#5975A4', saturation=1)
_, _ = plt.xticks(np.arange(0, 90, 5), np.arange(0, 90, 5))
plt.title('Number of Open Credit Lines')

Is there a difference in number of credit lines between fully paid loans and charged-off loans?

In [ ]:
loans.groupby('loan_status')['open_acc'].describe()

## 5.20 pub_rec

Data Dictionary: "Number of derogatory public records."

In [ ]:
loans['pub_rec'].value_counts().sort_index()

Is there a difference in average public records between fully paid loans and charged-off loans?

In [ ]:
loans.groupby('loan_status')['pub_rec'].describe()

## 5.21 revol_bal

Data Dictionary: "Total credit revolving balance."

In [ ]:
loans['revol_bal'].describe()

Do a log transform:

In [ ]:
loans['log_revol_bal'] = loans['revol_bal'].apply(lambda x: np.log10(x+1))

In [ ]:
loans.drop('revol_bal', axis=1, inplace=True)

In [ ]:
plot_var('log_revol_bal', 'Log Revolving Credit Balance', continuous=True)

In [ ]:
loans.groupby('loan_status')['log_revol_bal'].describe()

There isn't a large difference in the means.

## 5.22 revol_util

Data Dictionary: "Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit."

In [ ]:
loans['revol_util'].describe()

In [ ]:
plot_var('revol_util', 'Revolving Line Utilization', continuous=True)

In [ ]:
loans.groupby('loan_status')['revol_util'].describe()

## 5.23 total_acc

Data Dictionary: "The total number of credit lines currently in the borrower's credit file."

In [ ]:
plt.figure(figsize=(12,3), dpi=90)
sns.countplot(x= loans['total_acc'], order=sorted(loans['total_acc'].unique()), color='#5975A4', saturation=1)

_, _ = plt.xticks(np.arange(0, 176, 10), np.arange(0, 176, 10))
plt.title('Total Number of Credit Lines')

In [ ]:
loans.groupby('loan_status')['total_acc'].describe()

No large differences here.

## 5.24 initial_list_status

Data Dictionary: "The initial listing status of the loan. Possible values are – W, F." I'm not sure what this means.

In [ ]:
plot_var('initial_list_status', 'Initial List Status', continuous=False)

## 5.25 application_type

Data Dictionary: "Indicates whether the loan is an individual application or a joint application with two co-borrowers."

In [ ]:
loans['application_type'].value_counts()

Let's just compare the charge-off rates by application type:

In [ ]:
loans.groupby('application_type')['loan_status'].value_counts(normalize=True).loc[:,'Charged Off']

Joint loans are slightly less likely to be charged-off.

## 5.26 mort_acc

Data Dictionary: "Number of mortgage accounts."

In [ ]:
loans['mort_acc'].describe()

Not sure how someone can have 51 mortgage accounts...but apparently they do. Check the top 10 values:

In [ ]:
loans['mort_acc'].value_counts().head(10)

Compare the summary statistics by loan status:

In [ ]:
loans.groupby('loan_status')['mort_acc'].describe()

Individuals who pay off their loans are more likely to have several mortgage accounts.

## 5.27 pub_rec_bankruptcies

Data Dictionary: "Number of public record bankruptcies."

In [ ]:
loans['pub_rec_bankruptcies'].value_counts().sort_index()

In [ ]:
plot_var('pub_rec_bankruptcies', 'Public Record Bankruptcies', continuous=False)

# 6. More Pre-processing
<a id="6"></a>

## 6.1 Convert loan status to 0/1 charge-off indicator
<a id="6.1"></a>

Change the response variable `loan_status` to a 0/1 variable, where 0 indicates fully paid and 1 indicates charge-off:

In [ ]:
loans['charged_off'] = (loans['loan_status'] == 'Charged Off').apply(np.uint8)
loans.drop('loan_status', axis=1, inplace=True)

## 6.2 Create dummy variables
<a id="6.2"></a>

How many variables do we currently have?

In [ ]:
loans.shape

If any categorical variables have missing values, we'll need to create NaN dummy variables for those. So first check which variables have missing data:

In [ ]:
missing_fractions = loans.isnull().mean().sort_values(ascending=False) # Fraction of data missing for each variable

In [ ]:
print(missing_fractions[missing_fractions > 0]) # Print variables that are missing data

There are no categorical variables with missing values, and therefore we don't need any `NaN` dummy variables.

Create dummy variables for the categorical variables:

In [ ]:
print(loans.columns)

In [ ]:
loans = pd.get_dummies(loans, columns=['sub_grade', 'home_ownership', 'verification_status', 'purpose', 'addr_state', 'initial_list_status', 'application_type'], drop_first=True)

How many variables are there now?

In [ ]:
loans.shape

Check our data with the new dummy variables:

In [ ]:
loans.sample(5)

## 6.3 Train/test split
<a id="6.3"></a>

We'll make our modeling problem more realistic by performing the train/test split based on the month that the loan was funded. That is, we'll use loans funded on earlier dates to predict whether future loans will charge-off. The variable `issue_d` includes the month and year that the loan was funded.

In [ ]:
loans['issue_d'].sample(5)

Are there any missing values?

In [ ]:
loans['issue_d'].isnull().any()

No. Let's convert the issue dates to datetime objects:

In [ ]:
loans['issue_d'] = pd.to_datetime(loans['issue_d'])

In [ ]:
loans['issue_d'].sample(5)

The new datetime values are all on the first day of the month. Check the summary statistics of the issue dates:

In [ ]:
loans['issue_d'].describe()

There are only 124 unique issue dates over the 10-year period because we only have month/year information. In this particular dataset, the first loans were issued in June 2007, and the most recent loans were issued in September 2017. The busiest month was October 2014 with 33,699 loans funded in that month. What is the distribution of loans funded in each year?

In [ ]:
plt.figure(figsize=(6,3), dpi=90)
loans['issue_d'].dt.year.value_counts().sort_index().plot.bar(color='darkblue')
plt.xlabel('Year')
plt.ylabel('Number of Loans Funded')
plt.title('Loans Funded per Year')

We'll form the test set from the most recent 10% of the loans.

In [ ]:
loans_train = loans.loc[loans['issue_d'] <  loans['issue_d'].quantile(0.9)]
loans_test =  loans.loc[loans['issue_d'] >= loans['issue_d'].quantile(0.9)]

In [ ]:
loans_train.shape, loans_test.shape

Check that we properly partitioned the loans:

In [ ]:
print('Number of loans in the partition:   ', loans_train.shape[0] + loans_test.shape[0])
print('Number of loans in the full dataset:', loans.shape[0])

What is the test size?

In [ ]:
loans_test.shape[0] / loans.shape[0]

The partition looks good, so we can delete the original `loans` dataframe:

In [ ]:
del loans

Let's look at the summary statistics of the issue dates in the train and test sets:

In [ ]:
loans_train['issue_d'].describe()

In [ ]:
loans_test['issue_d'].describe()

The training set includes loans from June 2007 to June 2016. The test set includes loans from July 2016 to September 2017.

Now we need to delete the `issue_d` variable, because it was not available before the loan was funded.

In [ ]:
loans_train.drop('issue_d', axis=1, inplace=True)
loans_test.drop('issue_d', axis=1, inplace=True)

Now separate the predictor variables from the response variable:
Because the input dataset is really big,  I only take 10% DATA for training.  You can use 100% data.

In [ ]:

subsetloans_train=loans_train.sample(frac=0.1)

#y_train = loans_train['charged_off']
y_train = subsetloans_train['charged_off']

y_test = loans_test['charged_off']

In [ ]:
#X_train = loans_train.drop('charged_off', axis=1)
X_train = subsetloans_train.drop('charged_off', axis=1)
X_test = loans_test.drop('charged_off', axis=1)

In [ ]:
del loans_train, loans_test

# 7. Linear Dependence of Charge-off on the Predictors
<a id="7"></a>

On the training set, we compute the Pearson correlation, F-statistics and $p$ value of each predictor with the response variable `charged_off`.

In [ ]:
linear_dep = pd.DataFrame()

Pearson correlations:

In [ ]:
for col in X_train.columns:
    linear_dep.loc[col, 'pearson_corr'] = X_train[col].corr(y_train)
linear_dep['abs_pearson_corr'] = abs(linear_dep['pearson_corr'])

$F$-statistics:

In [ ]:
from sklearn.feature_selection import f_classif
for col in X_train.columns:
    mask = X_train[col].notnull()
    (linear_dep.loc[col, 'F'], linear_dep.loc[col, 'p_value']) = f_classif(pd.DataFrame(X_train.loc[mask, col]), y_train.loc[mask])

Sort the results by the absolute value of the Pearson correlation:

In [ ]:
linear_dep.sort_values('abs_pearson_corr', ascending=False, inplace=True)
linear_dep.drop('abs_pearson_corr', axis=1, inplace=True)

Reset the index:

In [ ]:
linear_dep.reset_index(inplace=True)
linear_dep.rename(columns={'index':'variable'}, inplace=True)

View the results for the top 20 predictors most correlated with `charged_off`:

In [ ]:
linear_dep.head(100)

The variables most linearly correlated with `charged_off` are the interest rate, loan period (term), FICO score, debt-to-income ratio, number of mortgages, income, the loan grade, and the loan amount.

Now view the results for the 20 least correlated predictors:

In [ ]:
linear_dep.tail(20)

It looks like the borrower's state of residence, the revolving balance, and several of the loan purposes are irrelevant for predicting charge-off.

# 8. Model Training and Testing
<a id="8"></a>

We implement machine learning pipelines consisting of one or more of the following steps, depending on the particular model:
1. Mean imputation of missing values
2. Dimension reduction using linear discriminant analysis (LDA)
3. Data standardization: rescaling to zero mean and unit variance
4. The chosen model

We will evaluate and compare the following models using a cross-validated AUROC score on the training set:
1. Logistic regression with SGD training
2. Random forest
3. k-nearest neighbors

We'll perform some hyperparameter tuning for each model to choose the most promising model, then more carefully tune the hyperparameters of the best-performing model.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV

## 8.1 Logistic regression with SGD training
<a id="8.1"></a>

The `SGDClassifier` estimator in scikit-learn implements linear classifiers (SVM, logistic regression, and others) with stochastic gradient descent (SGD) training. A particular linear classifier is chosen through the `loss` hyperparameter. Because we want to predict the probability of charge-off, we choose logistic regression (a probabilistic classifier) by setting `loss = 'log'`.

In [ ]:
from sklearn.linear_model import SGDClassifier

The machine learning pipeline:

In [ ]:
pipeline_sgdlogreg = Pipeline([
    ('imputer', SimpleImputer(copy=False)), # Mean imputation by default
    ('scaler', StandardScaler(copy=False)),
    ('model', SGDClassifier(loss='log_loss', max_iter=1000, tol=1e-3, random_state=1, warm_start=True))
])

A small grid of hyperparameters to search over:

In [ ]:
param_grid_sgdlogreg = {
    'model__alpha': [10**-6, 10**-5, 10**-2, 10**1, 10**2,10**3, 10**4],
    'model__penalty': ['l1', 'l2'],
 #   'model__bonus': [1,2,3,4,5,6,7,8,0]
}

Create the search grid object:

In [ ]:
grid_sgdlogreg = GridSearchCV(estimator=pipeline_sgdlogreg, param_grid=param_grid_sgdlogreg, scoring='roc_auc', n_jobs=1, pre_dispatch=1, cv=5, verbose=1, return_train_score=False)

Conduct the grid search and train the final model on the whole dataset:

In [ ]:
grid_sgdlogreg.fit(X_train, y_train)

Mean cross-validated AUROC score of the best model:

In [ ]:
grid_sgdlogreg.best_score_

Best hyperparameters:

In [ ]:
grid_sgdlogreg.best_params_

## 8.2 Support Vector Machine with SGD training. Dantong Provides an example on how to introduce a new machine learning model (Support Vector Machine or Neural Networks)  into the script.  You can customize your code to get a better performance.  You can use GridSearch Cross Valide to find the best parameter set.  

I removed the cell content from the following cell, you need to fill in the SVC related function call and make it work!  If you directly run them, you will get error message. You are required to make the following cell work!

In [ ]:
from sklearn.svm import SVC

pipeline_SVM = Pipeline([
    ('imputer', SimpleImputer(copy=False)), # Mean imputation by default
    ('scaler', StandardScaler(copy=False)),
    ('model', SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=1, warm_start=True))
])

param_grid_SVM = {
    'model__alpha': [10**-6, 10**-5, 10**-2, 10**1, 10**2, 10**3, 10**4],
    'model__penalty': ['l1', 'l2'],
}

# You can use the GridSearchCV over pipeline to get the best set of hyper-parameters!
#pipeline_SVM.classes_
grid_SVM = GridSearchCV(estimator=pipeline_SVM, param_grid=param_grid_SVM, scoring='roc_auc', n_jobs=1, pre_dispatch=1, cv=5, verbose=1, return_train_score=False)

In [ ]:
grid_SVM.fit(X_train, y_train)

In [ ]:
grid_SVM.best_score_

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV

# LinearSVC is much faster than standard SVC for a large, linearly separable dataset.
# We wrap it in CalibratedClassifierCV to get probability estimates needed for AUROC.
# Note: dual="auto" handles both small and large datasets efficiently.

pipeline_SVM = Pipeline([
    ('imputer', SimpleImputer(copy=False)),
    ('scaler', StandardScaler(copy=False)),
    # CalibratedClassifierCV wraps LinearSVC to enable predict_proba (required for AUROC)
    ('model', CalibratedClassifierCV(LinearSVC(random_state=1, max_iter=1000, dual="auto", tol=1e-3), cv=3, method='isotonic'))
])

param_grid_SVM = {
    # Tuning the regularization parameter C
    'model__estimator__C': [0.1, 1],
}

# The number of CV folds (cv) is reduced to 2 for a faster initial search on this slow model.
grid_SVM = GridSearchCV(estimator=pipeline_SVM, param_grid=param_grid_SVM,
                        scoring='roc_auc', n_jobs=1, cv=2, verbose=1, return_train_score=False)

print("Starting SVM Grid Search. This may take a while...")
grid_SVM.fit(X_train, y_train)

print("\nMean cross-validated AUROC score of the best SVM model:", grid_SVM.best_score_)
print("Best SVM hyperparameters:", grid_SVM.best_params_)

## 8.3 Random forest classifier
<a id="8.2"></a>

Next we train a random forest model. Note that data standardization is not necessary for a random forest.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensembles import CatboostClassifier

In [ ]:
# Random Forest Classifier pipeline
pipeline_rf = Pipeline([
    ('imputer', SimpleImputer(copy=False)),
    ('model', RandomForestClassifier(random_state=1, n_jobs=-1, n_estimators=500))
])

# Hyperparameter grid for Random Forest
param_grid_rf = {
    'model__n_estimators': [100, 1000],  # Number of trees in the forest
    'model__max_depth': [10, 20],      # Maximum depth of the tree
    'model__min_samples_split': [2, 5], # Minimum number of samples required to split an internal node
    'model__min_samples_leaf': [1, 2]   # Minimum number of samples required to be at a leaf node
}

# GridSearchCV for Random Forest
grid_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf, scoring='roc_auc',
                      n_jobs=-1, cv=5, verbose=1, return_train_score=False)


grid_rf.fit(X_train, y_train)

# Print the best score and best parameters
print("Best AUROC score:", grid_rf.best_score_)
print("Best parameters:", grid_rf.best_params_)


In [ ]:
help(RandomForestClassifier)

In [ ]:
pipeline_rfc = Pipeline([
    ('imputer', SimpleImputer(copy=False)),
    ('model', RandomForestClassifier(random_state=1, n_jobs=-1))
])

In [ ]:
param_grid_rfc = {
    'model__n_estimators': [500]
}

The random forest takes very long to train, so we don't test different hyperparameter choices. We'll still use `GridSearchCV` for the sake of consistency.

In [ ]:
grid_rfc = GridSearchCV(
    estimator=pipeline_rfc,
    param_grid=param_grid_rfc,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1,
    verbose=1,
    return_train_score=False
)

In [ ]:
grid_rfc.fit(X_train, y_train)

The AUROC will always improve (with decreasing gains) as the number of estimators increases, but it's not necessarily worth the extra training time and model complexity.

Mean cross-validated AUROC score of the random forest:

In [ ]:
grid_rfc.best_score_

Not quite as good as logistic regression, at least according to this metric.

## 8.4 k-nearest neighbors
<a id="8.3"></a>

Next we try k-nearest neighbors. We need to reduce the number of variables to 10 or fewer ([reference](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#Dimension_reduction)) for kNN to perform well. We'll use LDA for dimension reduction. The number of component variables to keep is a hyperparameter.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

In [ ]:
pipeline_knn = Pipeline([
    ("imputer", SimpleImputer(copy=False)),
    ("lda", LDA(n_components=1)),   # for 2 classes, max is 1
    ("model", KNeighborsClassifier())
])

In [ ]:
param_grid_knn = {
    "model__n_neighbors": [3, 5, 11, 21],
    "model__weights": ["uniform", "distance"],
    "model__p": [1, 2]   # Manhattan vs Euclidean distance
}

In [ ]:
grid_knn = GridSearchCV(
    estimator=pipeline_knn,
    param_grid=param_grid_knn,
    scoring="roc_auc",
    cv=5,
    n_jobs=-1,
    verbose=1,
    return_train_score=False
)

In [ ]:
grid_knn.fit(X_train, y_train)

Mean cross-validated AUROC score of the best model:

In [ ]:
grid_knn.best_score_

Best hyperparameters:

In [ ]:
grid_knn.best_params_

Only 3 LDA components are necessary for kNN to perform almost as well as logistic regression!

## 8.5 Tune hyperparameters on the chosen model more finely
<a id="8.4"></a>

The three models performed quite similarly according to the AUROC:

In [ ]:
print('Cross-validated AUROC scores')
print(grid_sgdlogreg.best_score_, '- Logistic regression')
print(grid_rfc.best_score_, '- Random forest')
print(grid_knn.best_score_, '- k-nearest neighbors')

Logistic regression squeaked out ahead, and coupled with the fact that `SGDClassifier` trains much faster than the other two models, we'll select logistic regression as our final model. Now we'll tune the hyperparameters more finely.

You can also try many other ensembles models. Please search the internet and find all ensemble models.

1.11. Ensemble methods
1.11.1. Bagging meta-estimator
1.11.2. Forests of randomized trees
1.11.3. AdaBoost
1.11.4. Gradient Tree Boosting
1.11.5. Histogram-Based Gradient Boosting
1.11.6. Voting Classifier
1.11.7. Voting Regressor
1.11.8. Stacked generalization

```
# This is formatted as code
```



You can also try deep neural networks (MLP)

In [ ]:
from sklearn.neural_network import MLPClassifier
print("MLPClassifier imported successfully.")

In [ ]:
pipeline_mlp = Pipeline([
    ('imputer', SimpleImputer(copy=False)), # Mean imputation by default
    ('scaler', StandardScaler(copy=False)),
    ('model', MLPClassifier(random_state=1, max_iter=1000))
])
print("MLP pipeline defined.")

param_grid_mlp = {
    'model__hidden_layer_sizes': [(50,), (100,), (50, 50)], # Single or double hidden layers with different neuron counts
    'model__activation': ['relu', 'tanh'],               # Activation function for the hidden layer
    'model__solver': ['adam', 'sgd'],                    # Solver for weight optimization
    'model__alpha': [0.0001, 0.001, 0.01],               # L2 regularization parameter
    'model__learning_rate_init': [0.001, 0.01]           # Initial learning rate
}
print("MLP hyperparameter grid defined.")

Mean cross-validated AUROC score of the best model:

In [ ]:
grid_sgdlogreg = GridSearchCV(
    estimator=pipeline_mlp,
    param_grid=param_grid_mlp,
    scoring='roc_auc',
    n_jobs=-1,
    cv=5,
    verbose=1,
    return_train_score=False
)

In [ ]:
grid_sgdlogreg.fit(X_train, y_train)

Best hyperparameters:

In [ ]:
grid_sgdlogreg.best_params_

By some coincidence, the optimal hyperparameters here are the same as from our first grid search for logistic regression!

## 8.6 Test set evaluation
<a id="8.5"></a>

Now we can finally see how our chosen model performs on the test data (the most recent 10% of the loans).

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
y_score = grid_sgdlogreg.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_score)

In [ ]:
y_score = grid_SVM.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_score)

In [ ]:
y_score = grid_SVM.predict(X_test)
print(y_score)
roc_auc_score(y_test, y_score)

In [ ]:
y_score = grid_knn.predict_proba(X_test)[:,1]

In [ ]:
y_score = grid_rfc.predict_proba(X_test)[:,1]

The test set AUROC score is somewhat lower than the cross-validated score (0.713).

# 9. Conclusion
<a id="9"></a>

We applied machine learning methods to predict the probability that a requested loan on LendingClub will charge off. After training and evaluating three different models (logistic regression, random forest, and k-nearest neighbors), we found that all three performed similarly according to a cross-validated AUROC score on the training data. We selected logistic regression (with ridge penalty) because it was the fastest model to train, and this model obtained an AUROC score of 0.689 on a test set consisting of the most recent 10% of the loans.

This model, while far from perfect, can provide a somewhat informed prediction of the likelihood that a loan will charge off, using only data available to potential investors before the loan is fully funded.

We also found that, according to linear measures of correlation between the predictors and the response, the most important variables for predicting charge-off are the loan interest rate and term, and the borrower's FICO score and debt-to-income ratio.